In [0]:
import numpy as np

In [0]:
sudoku1 = np.array([[8,7,6,9,0,0,0,0,0], 
                    [0,1,0,0,0,6,0,0,0], 
                    [0,4,0,3,0,5,8,0,0], 
                    [4,0,0,0,0,0,2,1,0],
                    [0,9,0,5,0,0,0,0,0],
                    [0,5,0,0,4,0,3,0,6], 
                    [0,2,9,0,0,0,0,0,8],
                    [0,0,4,6,9,0,1,7,3],
                    [0,0,0,0,0,1,0,0,4]])

In [198]:
# TEST CELL - test every little code you want to

#sudoku1[0][3]
#a = np.arange(1,10).reshape(-1, 1)
#np.sqrt(9)
#np.transpose(a)
#np.reshape(a,(9,1))
#a

#a = np.array((1,2,3))[np.newaxis, :]
#b = np.array((1,2,3))[:, np.newaxis]
#c = np.matmul(b,a)

#a = np.arange(0,9).reshape((3,3))
#a.reshape((9,1))
#a.transpose().reshape((9,1)) 
#b = np.hstack((a.reshape((9,1)), a.transpose().reshape((9,1))))
#b

#a = np.array((0,1,2))
#b = np.ones((3,3))
#c = np.array((0,9,18)).reshape(3,1)
#a*b + np.arange(0,20,9).reshape(-1,1)
#np.repeat([a],3, axis=0).flatten()
#np.repeat([a],3)

#wb=0
#hb=0
#r=3
#a = np.array((0,1,2))
#bx = np.repeat([a],r) + wb
#by = np.repeat([a],r, axis=0).flatten() + hb
#block_peers = np.hstack((bx,by))
#block_peers

#a = np.arange(0,20)
#np.where(a[1:1+r] == 2)[0][0]
#np.zeros((1,9+1)).transpose()

a= np.arange(1,20)
b = np.array((4,8,9))
a[b]

array([ 5,  9, 10])

In [0]:
# UPDATE: using ONE-HOT vectors of shape(1,10) for value_sets. First value is 1 if cell has fixed value
# also the corrsponding index is set to 1:
# Example: if cell has a fxed value of 5
# [1,0,0,0,0,1,0,0,0,0]
#  0 1 2 3 4 5 6 7 8 9 <- indexes: index equals value of cell. 0 indicates if fixed

def initialize(sudoku):
    h,w = sudoku.shape # using w as 9
    if h != w:
        print("Not square sudoku")
        #return
    new_sudoku = np.empty((h*w, w+1))
    #new_sudoku = np.empty((h*w, w+1+20)) # if peers are added at initialization
    for i in range(w): # ith row
        for j in range(w): # jth column
            
            if sudoku[i][j] == 0:
                value_set = np.ones((1,w+1))
                value_set[0][0] = 0
            else:
                value_set = np.zeros((1,w+1))
                value_set[0][0] = 1
                value_set[0][sudoku[i][j]] = 1
            new_sudoku[i*w+j] = value_set
            #new_sudoku[i*w+j] = np.concatanate(value_set, find_peers(i*w+j, w)) # if peers are added at initialization
    return new_sudoku

In [0]:
sudoku2 = initialize(sudoku1)
#sudoku2

In [0]:
def find_peers(index, range):
    h = index // range
    w = index % range
    r = np.sqrt(range) # for 3x3 matrix range=9, r=3
    wb = w // r
    hb = h // r # Block index - shows in which 3x3 block the cell is
    
    hx = np.full((range,1), h) # vertical 9x1 array
    hy = np.arange(0,range).reshape(-1, 1) # make it vertical
    row_peers = np.hstack((hx,hy)) # concatenate to vertical arrays
    #print(row_peers)
    
    wx = hy
    wy = np.full((range,1), w)
    column_peers = np.hstack((wx,wy)) # concatenate to vertical arrays
    #column_peers2 = column_peers[:,0]*range + column_peers[:,1]
    #print(column_peers2)
    
    a = np.array((0,1,2))
    bx = np.repeat([a],r, axis=0).reshape(range, 1) + hb*r
    by = np.repeat(a,r).reshape(-1, 1) + wb*r
    block_peers = np.hstack((bx,by)) # concatenate to vertical arrays
    #print(block_peers)
    
    peers = np.unique(np.vstack((row_peers, column_peers, block_peers)),axis=0)
    #print(np.where((peers[:,0] == h) * (peers[:,1] == w))[0][0]) # Finds the index of the row where the cell is located
    peers = np.delete(peers, np.where((peers[:,0] == h) * (peers[:,1] == w))[0][0], axis=0)
    peers_1D = np.int8(peers[:,0]*range + peers[:,1])
    
    return peers_1D

def constraint_propagation(sudoku):
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1 # First column refers to fixed value. IMPORTANT: if the initialize funtion is changed so that the sudoku matrix stores peers, this value need to be redefined
    change = 1
    while change == 1: # go through the loop until there has been zero changes:
        change = 0
        for i in range(c):
            peers = find_peers(i, r) # Always same output, would be okay to store in a variable. Maybe it should be added in the sudoku initialization.
            if sudoku[i][0] == 1: # Cell value is fixed
                value = np.where(sudoku[i][1:1+r] == 1)[0][0]+1
                for p in peers:
                    if sudoku[p][0] == 0 and sudoku[p][value] == 1: # if peer is not fixed, remove cell's value from its value_set
                        sudoku[p][value] = 0
                        if np.sum(sudoku[p][1:1+r]) == 1:
                            sudoku[p][0] = 1 # if peers value_set has decreased to 1 value, declare peer fixed
                        change = 1
                    elif sudoku[p][0] == 1: # if peer is fixed
                        peer_value = np.where(sudoku[p][1:1+r] == 1)[0][0]+1
                        if sudoku[i][peer_value] == 1: # if peers value is in cells value_set
                            sudoku[i][peer_value] = 0 # remove peers value from cells value_set
                            if np.sum(sudoku[i][1:1+r]) == 1:
                                sudoku[i][0] = 1 # if cells value_set has decreased to 1 value, declare cell fixed
                            change = 1
                
    return sudoku

def output_9x9(sudoku):
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1
    out_sudoku = np.empty(c)
    for i in range(c):
        if sudoku[i][0] == 1:
            out_sudoku[i] = np.where(sudoku[i][1:1+r] == 1)[0][0]+1
        else:
             out_sudoku[i] = 0
    return out_sudoku.reshape((r,r))
        
def check_if_solved(sudoku):
    checksum = np.unique(np.arange(1,10))
    c, r = sudoku.shape # cell index, range of values for each sell
    r = r-1
    R = np.int(np.sqrt(r))
    
    for i in range(r):
        row =    i*r + np.arange(0,r,1)
        column = i + np.arange(0, r*r, r)
        block =  (i//R*R*r + i%R*R) + (np.arange(0,R,1, dtype=np.int_) * np.ones((R,R),dtype=np.int_) + np.arange(0,R*r,r, dtype=np.int_).reshape(-1,1)).flatten() # 0, 1, 2 for 9x9 sudoku * 3x3 ones
        if np.sum(sudoku[row]) != 18 or np.sum(sudoku[column]) != 18 or np.sum(sudoku[block]) != 18:
            return False
    return True
        

In [266]:
sudoku3 = constraint_propagation(sudoku2)
#sudoku2.shape
#sudoku4 = output_9x9(sudoku3)
check_if_solved(sudoku3)

True

In [193]:
sudoku4

array([[8., 7., 6., 9., 1., 4., 5., 3., 2.],
       [3., 1., 5., 2., 8., 6., 7., 4., 9.],
       [9., 4., 2., 3., 7., 5., 8., 6., 1.],
       [4., 3., 8., 7., 6., 9., 2., 1., 5.],
       [6., 9., 1., 5., 2., 3., 4., 8., 7.],
       [2., 5., 7., 1., 4., 8., 3., 9., 6.],
       [1., 2., 9., 4., 3., 7., 6., 5., 8.],
       [5., 8., 4., 6., 9., 2., 1., 7., 3.],
       [7., 6., 3., 8., 5., 1., 9., 2., 4.]])